# Import all SET metadata into MNE object

### ToC
- Import the full Matlab EEG.event struct containning `epoch_id`
- Import SET with MNE (epoched data, basic event structure)
    - Use SciPy's loadmat() to read the struct and make a pd.df ([source](https://github.com/mne-tools/mne-python/issues/3837#issuecomment-266460434) found [here](https://mne.discourse.group/t/creating-epochs-with-variable-event-latencies/3853)).
- Look into the true event df to find epochs falling into set times

### Imports

In [1]:
import mne
import os
from scipy.io import loadmat

import numpy as np
import pandas as pd

Define path to files to work with 

In [5]:
path = "../hyper_cleaned/samples/"
subject_1 = "SAMPLE-1_ES_epochs.set"
subject_2 = "SAMPLE-2_ES_epochs.set"

### Making a custom event df

Because MNE's function `read_epochs_eeglab()` does all the work of formating the SET file's events df into a MNE-formated df, we lose important information (here, the column `epoch_id`). The class we define bellow allow to keep all these metadata and provide tools to 'cut' it to specific periodes (i.e., a time window) dynamicaly (i.e., a sliding window!). 

Bellow we import a `class` that:
- takes a SET file and read the (full) set of metadata associated to it (see `Class.df`)
- create a `Class.df_sw` which only contains events from `Class.df` which falls between `sw_min` and `sw_max`.
    - if `sw_min` and `sw_max` are defined (floats)
    - or leave `Class.df_sw` as an empty pd.Dataframe if `sw_min` and `sw_max` are `None`

> The function `read_set_events()` that enables reading MatLab's EEG.event struct embeded into a SET file into a panda df is sourced from [here](https://github.com/mne-tools/mne-python/issues/3837#issuecomment-266460434) and found [here](https://mne.discourse.group/t/creating-epochs-with-variable-event-latencies/3853).

Import custom class

In [6]:
from utils_eeg_sw import import_set_custom

Instantiate the class twice for 2 subject 

In [7]:
sub1 = import_set_custom(path+subject_1, sw_min=None, sw_max=None)
sub2 = import_set_custom(path+subject_2, sw_min=None, sw_max=None)

In [8]:
sub1.event_slider(sw_min=2, sw_max=16)

# new_event = sub1.convert_to__MNE_event()

# type(new_event)

In [9]:
sub2.event_slider(2.0, 14.0)
sub2.df_sw

,epoch,type,duration,epoch_id,latency,urevent
0,5,chan25,1,5,1,5
1,6,chan25,1,6,501,6
2,7,chan25,1,7,1001,7
3,8,chan25,1,8,1501,8
4,9,chan25,1,9,2001,9
5,10,chan25,1,10,2501,10
6,11,chan25,1,11,3001,11
7,12,chan25,1,12,3501,12
8,13,chan25,1,13,4001,13
9,14,chan25,1,14,4501,14


## Bellow this point, it's just tries and fails

In [ ]:
epochs = mne.io.read_epochs_eeglab(path+sub1)

In [ ]:
def read_set_events(filename, ignore_fields=None):
	'''Open set file, read events and turn them into a dataframe
	Parameters
	----------
	filename: str
		Name of the set file to read (absolute or relative path)
	ignore_fields: list of str | None
		Event fields to ignore
	Returns
	-------
	df: pandas.DatFrame
		Events read into a dataframe
	'''
	EEG = loadmat(filename, uint16_codec='latin1',
				  struct_as_record=False, squeeze_me=True)['EEG']
	flds = [f for f in dir(EEG.event[0]) if not f.startswith('_')]
	events = EEG.event
	df_dict = dict()
	for f in flds:
		df_dict[f] = [ev.__getattribute__(f) for ev in events]
	df = pd.DataFrame(df_dict)

	# reorder columns:
	take_fields = ['epoch', 'type']
	ignore_fields = list() if ignore_fields is None else ignore_fields
	take_fields.extend([col for col in df.columns if not
					 (col in take_fields or col in ignore_fields)])
	return df.loc[:, take_fields]

In [ ]:
df = read_set_events(path+sub1)
df

In [ ]:
    

# epochs1 = mne.io.read_epochs_eeglab(path+sub1)
# epochs2 = mne.io.read_epochs_eeglab(path+sub2)

x=0
shapes=list()

# iterate over files in path
for filename in os.listdir(path):
    if filename[-3:] == "set" :
        f = os.path.join(path, filename)
        epochs = mne.io.read_epochs_eeglab(f)
        shapes.append(np.shape(epochs._data))
    else:
        pass



In [ ]:
print(np.mean(shapes, axis=0))
print("Data left after pre-processing (%): ")
print(((np.mean(shapes, axis=0)[0]*100)/(5*60)))

In [ ]:
shapes[2]

In [ ]:
print(np.shape(epochs1.selection))
print(epochs1.selection)

In [ ]:
epochs1.plot(block=True, n_channels=24)